<a href="https://colab.research.google.com/github/bernardobdp/SIM_YAC_PRUDENTE_UPSE2023/blob/main/Prueba3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
import math

In [19]:
L=10000
dx=2500
Tiempo=100000
Dt=1
k=50 # permeabilidad en milidarci
por=0.2 #porosidad
vis=1 #viscocidad centipoise
Beta=1
CompTotal=1**(-6) #ps1-1 compresibilidad
A=200000 #ft^2


In [20]:
Alpha=k/(vis*por*CompTotal)
Alpha
Eta=(Alpha*Dt)/(dx**2)
Eta

4e-05

In [21]:
#ESQUEMA EXPLICITO

In [22]:

def Presdt(P0,P1,P2,Eta):
  Pdt=P1+Eta*(P0-2*P1+P2)
  return(Pdt)

Pexplicita=np.zeros((Tiempo//Dt,L//dx))#division exacta para las colummnas


In [23]:
#Condiciones iniciales
Pyac=1000
Piz=2000
Pder=0
Pin=np.ones(L//dx)
Pres=np.zeros(L//dx)
Pin=Pin*Pyac
Pin[0]=Piz
Pin[len(Pin)-1]=Pder
Pin

array([2000., 1000., 1000.,    0.])

In [24]:
t=Dt
Pexplicita[0]=Pin
Pexplicita.shape
while t<Tiempo:#ciclo de tiempo
  for i in range(0,len(Pin)-1): #ciclo de espacio 
    if i==0:
      Pres[i]=Pin[0]
    elif i==len(Pin)-1:
      Pres[i]=Pin[len(Pin)-1]
    else:
      Pres[i]=Presdt(Pin[i-1],Pin[i],Pin[i+1],Eta)
  Pexplicita[t//Dt]=Pres
  t=t+Dt
  Pin=Pres 
#solucion explicita solucion con un sistema de ecuaciones


In [25]:
Pexplicita

array([[2000.        , 1000.        , 1000.        ,    0.        ],
       [2000.        , 1000.04      ,  999.96      ,    0.        ],
       [2000.        , 1000.0799952 ,  999.9200064 ,    0.        ],
       ...,
       [2000.        , 1333.33146859,  666.66889757,    0.        ],
       [2000.        , 1333.33146883,  666.66889732,    0.        ],
       [2000.        , 1333.33146907,  666.66889706,    0.        ]])

In [26]:
#ESQUEMA IMPLICITA

In [27]:
Pimplicita=np.zeros((Tiempo//Dt,L//dx))

Pin=np.ones(L//dx)
Pres=np.zeros(L//dx)
Pin=Pin*Pyac
Pin[0]=Piz
Pin[len(Pin)-1]=Pder
N=L//dx
Matriz_resolucion=np.zeros([N-2,N-2])


In [28]:
for i in range(N-2): #Matriz de resolucion 
  Matriz_resolucion[i,i]=1+2*Eta #porque la diagonal de los indices son iguales a i y j posion de la diagonal
  if i>0:
    Matriz_resolucion[i,i-1]=-Eta # llena todas las casillas de la izquierda
  if i<(N-3):
    Matriz_resolucion[i,i+1]=-Eta #llena todas las casillas de la derecha
   

In [29]:
Pcal=np.zeros([N-2])
Pcal=np.linalg.solve(Matriz_resolucion,Pin[1:-1])

In [30]:
Pcal

array([999.9600016, 999.9600016])

In [31]:
Pres=np.concatenate(([Piz],Pcal,[Pder]))
Pres

TypeError: ignored

In [ ]:
Pimplicita[0]=Pin
t=Dt
while t<Tiempo: #ciclo en el tiempo
    Pcal=np.linalg.solve(Matriz_resolucion,Pin[1:-1])
    Pres=np.concatenate(([Piz],Pcal,[Pder]))
    Pimplicita[t//Dt]=Pres
    Pin=Pres
    t=t+Dt

ValueError: ignored